### Import

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from local_library.image_operations import *
from local_library.ellipsoid import *
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import time

### Configuration

select image

In [ ]:
# Directory for seedling images
FOLDER_DIRECTION = 'D:/#Work/4-1 WiL/Workspace/Dataset/seedling_pi_webcam_1/EcRfSd_03_3/'
if FOLDER_DIRECTION[-2] == '4':
    tray_orientation = 'LEFT'
    
# Thresholds for mean Y-value in image selection for different processes
QR_DETECTION_IMAGE_Y_MEAN_THRESHOLD  = 0.05
SEEDLING_STATUS_UPDATE_IMAGE_Y_MEAN_THRESHOLD  = 0.35


# Blue mark detection
# Blue mark color characteristics in YCbCr space
BLUE_MARK_COLOR_MEAN  = np.array([0.11555396, -0.06690868])
BLUE_MARK_COLOR_COVARIANCE  = np.array([[8.34169839e-04, -7.15137086e-05],
                                        [-7.15137086e-05,  1.78775272e-04]])
# Maximum pixel count and cluster count for blue mark detection before skipping
BLUE_MARK_MAX_PIXEL_THRESHOLD  = 800
BLUE_MARK_MAX_CLUSTER_THRESHOLD  = 10
# Threshold for determining blue mark probability
BLUE_MARK_PROBABILITY_THRESHOLD  = 0.1
# Reference shape distances and shape ratio for mark detection
MARK_REFERENCE_SHAPE_DISTANCES  = np.array([ 29, 54.5, (54.5 ** 2 + 29 ** 2) ** 0.5])
# min and max ratio thresholds for acceptable shape
MIN_MARK_SHAPE_RATIO_THRESHOLD  = ((MARK_REFERENCE_SHAPE_DISTANCES  - 2) / MARK_REFERENCE_SHAPE_DISTANCES.min()).repeat(2)
MAX_MARK_SHAPE_RATIO_THRESHOLD  = ((MARK_REFERENCE_SHAPE_DISTANCES  + 2) / MARK_REFERENCE_SHAPE_DISTANCES.min()).repeat(2)

processing

In [ ]:
# Tray dimensions and structure
TRAY_ROW_COUNT = 8                  # Number of rows in the seedling tray
TRAY_COLUMN_COUNT = 15              # Number of columns in the seedling tray
TRAY_HEIGHT = 290                   # Tray height in millimeters
TRAY_WIDTH = 545                    # Tray width in millimeters

# Number of Gaussian models used for probability calculations and Gaussian Mixture Model (GMM) 
MIN_NUM_GAUSSIAN_MODELS = 2         # Minimum number of Gaussian models per cell
MAX_NUM_GAUSSIAN_MODELS = 3         # Maximum number of Gaussian models per cell

# Image shape definitions based on tray dimensions
IMAGE_HEIGHT = TRAY_ROW_COUNT * 30 
IMAGE_WIDTH = TRAY_COLUMN_COUNT * 30
IMAGE_SHAPE = np.array([IMAGE_HEIGHT, IMAGE_WIDTH])
CELL_HEIGHT, CELL_WIDTH = IMAGE_HEIGHT // TRAY_ROW_COUNT, IMAGE_WIDTH // TRAY_COLUMN_COUNT
CELL_SHAPE = np.array([CELL_HEIGHT, CELL_WIDTH])
HALF_CELL_HEIGHT, HALF_CELL_WIDTH = CELL_HEIGHT // 2, CELL_WIDTH // 2
HALF_CELL_SHAPE = CELL_SHAPE // 2
PIXELS_TO_REAL_SIZE_RATIO = (np.array([TRAY_HEIGHT, TRAY_WIDTH]) / IMAGE_SHAPE).prod()

# Initial confidence values and scaling factors used in Bayesian segmentation
INIT_CONFIDENCE = 0.1                           # Minimum initial confidence for seedling identification
SPATIAL_BACKGROUND_PRIOR_SCALING = 1.5          # Scaling factor to prevent division by zero in Bayesian segmentation

# Threshold values for seedling probability and model updates
MEAN_SEEDLING_COLOR_CONFIDENCE_THRESHOLD = 0.01             # Threshold for color confidence; exceeding this means reliable seedling color
MIN_SEEDLING_PIXEL_IN_IMAGE_FOR_CALCULATION = 10            # Minimum number of seedling pixels required for calculation
SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD = 0.01                # Threshold ratio for seedling size relative to cell size
PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL = 0.1          # Probability threshold for updating the color model

# Covariance scaling factors for prior probability in the spatial domain
INIT_COVARIANCE_PRIOR_SCALING = 0.025                               # Covariance scaling factor for initial seedling position
COVARIANCE_POSITION_UPDATE_COLOR_CONFIDENCE_SCALING = 1             # Covariance scaling factor for position weight in color confidence updating
COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING = 0.08                   # Covariance scaling factor for all seedling cells
# COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING = 0.5                    # Covariance scaling factor for all seedling cells
COVARIANCE_PRIOR_EMPTY_CELL_SCALING = 0.004                         # Covariance scaling factor for empty cells
MAX_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING = 0.007             # Maximum allowed scaling for covariance of seedling cells
MIN_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING = 0.0001            # Minimum allowed scaling for covariance of seedling cells
COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING_RATIO = 2.0             # Scaling ratio for adjusting seedling cell covariance

# Weights and learning rates for updating seedling status confidence
BACKGROUND_CONFIDENCE_WEIGHT = 0.99                         # Weight for background confidence updates
PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT = 0.7            # Weight for updating seedling color confidence based on probability
POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT = 0.55              # Weight for updating seedling color confidence based on seedling position
# STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.7                # Weight for updating seedling status confidence based on seedling status
# COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.55                # Weight for updating seedling status confidence based on seedling color model
# POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.55             # Weight for updating seedling status confidence based on seedling position
STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.8                # Weight for updating seedling status confidence based on seedling status
COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.6                 # Weight for updating seedling status confidence based on seedling color model
POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.6              # Weight for updating seedling status confidence based on seedling position


SEEDLING_STATUS_CONFIDENCE_THRESHOLD = 0.9

### Select image

group image name

In [ ]:

# Get and sort list of image names
image_name_list = os.listdir(FOLDER_DIRECTION)
image_name_list.sort()

# Dictionary to store grouped images by timestamp
grouped_images = {}

# Group images by timestamp
for image_name in image_name_list:
    timestamp = '_'.join(image_name.split('_')[3:5])  # Extract timestamp (date and hour)
    if timestamp not in grouped_images:
        grouped_images[timestamp] = []
    grouped_images[timestamp].append(image_name)

# Dictionary to store the filtered images by hour
filtered_data = {}

# Iterate over grouped images to filter by hour
for timestamp, images in grouped_images.items():
    # Extract date and hour (YYYY-MM-DD_HH)
    date_hour = timestamp[:13]
    
    # # if int(date_hour[-2:]) in hour_selected or date_hour == '2024-11-05_13':
    # if (int(date_hour[-2:]) in hour_selected and int(date_hour[-5:-3])%2 == 1)or date_hour == '2024-11-05_13':

    # Check if this date_hour is already in filtered_data
    if date_hour in filtered_data:
        # Compare by number of images and by timestamp to keep the appropriate key
        current_timestamp, current_images = filtered_data[date_hour]
        if len(images) > len(current_images) or (len(images) == len(current_images) and timestamp < current_timestamp):
            filtered_data[date_hour] = (timestamp, images)
    else:
        # Store the first occurrence of this date_hour
        filtered_data[date_hour] = (timestamp, images)

# Rebuild the final dictionary with filtered results
grouped_images = {key: value for key, value in filtered_data.values()}

print(grouped_images)

select image from brightness

In [ ]:
timestamp_list = list(grouped_images.keys())
num_timestamp = len(timestamp_list)
update_seedling_status_image_name_list, qr_detection_image_name_list, blue_mark_detection_image_name_list = [], [], []
for timestamp in timestamp_list:
    num_image_per_timestamp = len(grouped_images[timestamp])
    y_mean_buffer = np.zeros(num_image_per_timestamp)
    for image_index in range(num_image_per_timestamp):
        rgb_image = plt.imread(f'{FOLDER_DIRECTION}{grouped_images[timestamp][image_index]}')[:,:,:3]
        image_shape = rgb_image.shape
        y_mean_buffer[image_index] = (rgb_image * np.array([0.299, 0.587, 0.114])).sum(axis=-1)[:,image_shape[1]//3:-image_shape[1]//3].mean()
    update_seedling_status_image_index = np.abs(y_mean_buffer - SEEDLING_STATUS_UPDATE_IMAGE_Y_MEAN_THRESHOLD ).argmin()
    update_seedling_status_image_name_list.append(grouped_images[timestamp][update_seedling_status_image_index])
    qr_detection_image_index = np.abs(y_mean_buffer - QR_DETECTION_IMAGE_Y_MEAN_THRESHOLD).argmin()
    qr_detection_image_name_list.append(grouped_images[timestamp][qr_detection_image_index])


read image, detect QR code and adjust perspective and crop image.

In [ ]:
resized_rgb_images_list = []

# plt.figure(figsize=(25, 150))
for image_index in range(num_timestamp):
# for image_index in range(2):  
    
    qr_detection_img = plt.imread(f'{FOLDER_DIRECTION}{qr_detection_image_name_list[image_index]}')[:,:,:3]
    rgb_img = plt.imread(f'{FOLDER_DIRECTION}{update_seedling_status_image_name_list[image_index]}')[:,:,:3]
    
    qr_detection_result = decodeQR(qr_detection_img)
    if len(qr_detection_result) > 0:
        tray_id = qr_detection_result[0].data.decode("utf-8").split('.')[0]
        tray_orientation = qr_detection_result[0].orientation
    
    ycbcr_img = convertRGBtoYCbCr(rgb_img)
    ycbcr_mean = ycbcr_img.reshape(-1,3).mean(0)
    blue_mark_color_mean_offset = np.array([
        (ycbcr_mean[1] - 0.006) * 2 if ycbcr_mean[1] > 0.006 else 0,
        (ycbcr_mean[2] - 0.002) * 0.05 if ycbcr_mean[2] < -0.002 else 0
    ])

    blue_mark_probability = calculateMultiGaussianDist(ycbcr_img[:,:,1:], BLUE_MARK_COLOR_MEAN  + blue_mark_color_mean_offset, BLUE_MARK_COLOR_COVARIANCE )
    blue_mark_probability /= blue_mark_probability.max()
    
    blue_mark_mask_position = np.stack(np.where(blue_mark_probability > BLUE_MARK_PROBABILITY_THRESHOLD ), axis = -1)
        
        
    corner_tray_found = False
    
    if len(blue_mark_mask_position) < BLUE_MARK_MAX_PIXEL_THRESHOLD  :
        labels = DBSCAN(eps = 5, min_samples = 5).fit_predict(blue_mark_mask_position)
        # Calculate the number of distinct seedlings
        number_of_mark = labels.max() + 1
        
        if number_of_mark < BLUE_MARK_MAX_CLUSTER_THRESHOLD  :
            blue_mark_position = np.array([
                blue_mark_mask_position[labels == index].mean(0) for index in range(number_of_mark)
            ])
            
            # Calculate the distance matrix between points
            mark_distance_matrix = calculateEuclidean(blue_mark_position[:, np.newaxis, :], blue_mark_position[np.newaxis, :, :])
            corner_tray_found = False

            # Loop through unique combinations of four points
            for point_a_index in range(len(blue_mark_position) - 3):
                if corner_tray_found: break
                for point_b_index in range(point_a_index + 1, len(blue_mark_position) - 2):
                    if corner_tray_found: break
                    for point_c_index in range(point_b_index + 1, len(blue_mark_position) - 1):
                        if corner_tray_found: break
                        for point_d_index in range(point_c_index + 1, len(blue_mark_position)):
                            # Select indices and calculate distances
                            index = np.array([point_a_index, point_b_index, point_c_index, point_d_index])
                            selected_distances = mark_distance_matrix[np.ix_(index, index)]
                            sorted_distances = np.sort(selected_distances.reshape(-1))[4::2]
                            # avg_distances = sorted_distances.reshape(3, 2).mean(axis=1)
                            
                            # Calculate shape ratio and check within range
                            sorted_distances_norm = sorted_distances / sorted_distances.min()
                            if np.all((MIN_MARK_SHAPE_RATIO_THRESHOLD  < sorted_distances_norm) & (sorted_distances_norm < MAX_MARK_SHAPE_RATIO_THRESHOLD )):
                                
                                # Determine corner quadrants
                                relative_positions = (blue_mark_position[index] - blue_mark_position[index].mean(0)).clip(-1, 1).astype(int)
                                corner_quadrant_ref_order = np.array([[-1, -1], [-1, 1], [1, 1], [1, -1]])
                                
                                # Adjust based on tray orientation
                                rotation_shift = {'UP': 1, 'LEFT': 2, 'DOWN': 3}.get(tray_orientation, 0)
                                corner_quadrant_ref_order = np.roll(corner_quadrant_ref_order, shift=rotation_shift, axis=0)
                                
                                # Map quadrants to order and extract corner positions
                                order = np.argmax((corner_quadrant_ref_order[:, None] == relative_positions).all(-1), axis=0)
                                
                                # tray_corner_position_l = tray_corner_position
                                
                                tray_corner_position = blue_mark_position[index][order]
                                
                                # print(f'{update_seedling_status_image_name_list[image_index]} : {(tray_corner_position_l - tray_corner_position).mean()}')
                                
                                corner_tray_found = True
                                break
    if corner_tray_found == False and image_index == 0:
        print(f'first tray cannot detect blue mark')
        blue_mark_probability == blue_mark_mask_position
        break
    
    rgb_image_roi = warpPerspectiveTransformation(rgb_img, tray_corner_position[:,::-1], IMAGE_HEIGHT, IMAGE_WIDTH)
    resized_rgb_images_list.append(rgb_image_roi)
        
    # plt.subplot(30,3,image_index+1), plt.imshow(rgb_image_roi)
    # plt.title(f'{update_seedling_status_image_name_list[image_index]}')
            
ycbcr_images_list = [convertRGBtoYCbCr(image) for image in resized_rgb_images_list]
ycbcr_vectors_list = [ image.reshape(-1, image.shape[-1]) for image in ycbcr_images_list]
number_of_images = len(timestamp_list)

### Declare variables for storing data

In [ ]:
# Initialize arrays to hold various data for each image in the dataset

# Array to store the status of seedlings in each cell for each image
seedling_status_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Arrays to store mean positions and covariance matrix of seedlings for each cell in each image
seedling_position_model_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2))
seedling_position_model_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2, 2))

# Array to store the number of Gaussian models used for each image
number_of_color_models_buffer = np.zeros(number_of_images, dtype=int)

# Arrays to store means and covariances of color models for each image
color_model_means_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3))
color_model_covariances_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3, 3))

# Arrays to store means and covariances of seedling color models for each cell in each image
seedling_color_model_each_cell_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3))
seedling_color_model_each_cell_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3, 3))

# Arrays to store ellipse sizes, seedling sizes (sum of probabilities), and size-to-cell ratios for each cell in each image
ellipse_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
seedling_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
seedling_size_to_cell_ratio_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Array to store the Euclidean distance between seedling center and planting cell center for each cell in each image
distance_position_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
# Array to store the Euclidean distance between mean Cb-Cr of each seedling and mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
distance_color_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Array to store confidence in seedling status for each cell in each image
seedling_status_confidence_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

### Visualization

In [ ]:
# plt.figure(figsize=(30,150))
# for i in range(number_of_images):
#     plt.subplot(30,5,1+i), plt.imshow(resized_rgb_images_list[i]), plt.title(timestamp_list[i].split(".")[0])
# fig = plt.figure(figsize=(20,10))
# for i in range(number_of_images):
#     ax = fig.add_subplot(7,5, 1 + i, projection="3d")
#     ax.scatter3D(ycbcr_vectors_list[i][:, 2], ycbcr_vectors_list[i][:, 1], ycbcr_vectors_list[i][:, 0], color="green", marker="o", alpha=0.2, s=2)
#     ax.set_title(timestamp_list[i])
#     ax.axes.set_xlim3d(left=-0.5, right=0.5)
#     ax.axes.set_ylim3d(bottom=-0.5, top=0.5)
#     ax.axes.set_zlim3d(bottom=0, top=1)
#     ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y")
#     ax.view_init(elev=7, azim=-7)

In [ ]:
# fig = plt.figure(figsize=(30,150))
# for i in range(number_of_images):
#     ax = fig.add_subplot(30,5, 1 + i, projection="3d")
#     ax.scatter3D(ycbcr_vectors_list[i][::10, 2], ycbcr_vectors_list[i][::10, 1], ycbcr_vectors_list[i][::10, 0], color="green", marker="o", alpha=0.2, s=2)
#     ax.set_title(timestamp_list[i])
#     ax.axes.set_xlim3d(left=-0.5, right=0.5)
#     ax.axes.set_ylim3d(bottom=-0.5, top=0.5)
#     ax.axes.set_zlim3d(bottom=0, top=1)
#     ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y")
#     ax.view_init(elev=7, azim=-7)

## Identifying Seedling Color

### Init background color

In [ ]:
# Day01 = Background
current_image_index = 0

# Extract the vector of background colors for Day 1 from the list of YCbCr vectors
background_color_vector = ycbcr_vectors_list[current_image_index]

# Calculate the mean and covariance matrix for the background color model
init_mean_background_color, init_covariance_background_color = calculateMeanAndCovariance(background_color_vector)

# Calculate Mahalanobis distances for each background color value from the background color model
background_color_mahalanobis_distances = calculateMahalanobis(background_color_vector, init_mean_background_color, init_covariance_background_color)

# Find the maximum Mahalanobis distance among the background color distances
max_background_color_mahalanobis_distance = np.max(background_color_mahalanobis_distances)

print('Mean of background :', init_mean_background_color)
print('Covariance matrix of background :\n', init_covariance_background_color)
print('Max distance of background color :', max_background_color_mahalanobis_distance)

### Update seedling color confidence

In [ ]:
# Initialize variables for updating the confidence that each pixel's color indicates a seedling
seedling_color_confidence = np.zeros(IMAGE_SHAPE)    

# Create position weight for target positions in the spatial domain to find seedling color vectors
position_weight_update_color_confidence = createPrior(COVARIANCE_POSITION_UPDATE_COLOR_CONFIDENCE_SCALING, IMAGE_SHAPE, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
position_weight_update_color_confidence = (position_weight_update_color_confidence / position_weight_update_color_confidence.max()) * (2 * POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT - 1) + (1 - POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT)

# Visualization
fig = plt.figure(figsize=(30,300))
# fig = plt.figure(figsize=(30,9))

current_image_index += 1
while( True ):
    
    # Extract the image and color vector for the current image
    current_image = ycbcr_images_list[current_image_index]
    current_image_color_vector = ycbcr_vectors_list[current_image_index]

    # Calculate Mahalanobis distances for each current image color value from the background color model
    current_image_mahalanobis_distances = calculateMahalanobis(current_image_color_vector, init_mean_background_color, init_covariance_background_color)

    # Visualization
    # Identify indices of color values in the current image that are significantly different from the background color model
    seedling_indices = np.where(current_image_mahalanobis_distances > max_background_color_mahalanobis_distance)
    # Extract the color vectors corresponding to the identified seedling indices
    seedling_color_vector = current_image_color_vector[seedling_indices]
    # Identify indices of color values in the current image that are significantly different from the background color model
    background_indices = np.where(current_image_mahalanobis_distances < max_background_color_mahalanobis_distance)
    # Extract the color vectors corresponding to the identified seedling indices
    background_color_vector = current_image_color_vector[background_indices]
    plt.subplot(100, 7, 1 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index]), plt.title(timestamp_list[current_image_index])
    ax = fig.add_subplot(100, 7, 2 + (7*(current_image_index-1)), projection="3d")
    ax.scatter3D(seedling_color_vector[:, 2], seedling_color_vector[:, 1], seedling_color_vector[:, 0], color="green", marker="o", alpha=0.2, s=2)
    ax.scatter3D(background_color_vector[:, 2], background_color_vector[:, 1], background_color_vector[:, 0], color="black", marker="o", alpha=0.2, s=2)
    ax.set_title('scatter plot'), ax.axes.set_xlim3d(left=-0.5, right=0.5), ax.axes.set_ylim3d(bottom=-0.5, top=0.5), ax.axes.set_zlim3d(bottom=0, top=1), ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y"), ax.view_init(elev=7, azim=-7)
    
    seedling_mask = np.where(current_image_mahalanobis_distances.reshape(IMAGE_SHAPE) > max_background_color_mahalanobis_distance, 1, 0)
    seedling_color_probability = np.where(seedling_mask > 0.5, PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT, 1 - PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT)
    
    position_weight = np.where(seedling_mask == 1, position_weight_update_color_confidence, 0.5)
    
    seedling_color_confidence = np.where(((seedling_color_confidence < INIT_CONFIDENCE).astype(int) * (seedling_mask == 1).astype(int)), INIT_CONFIDENCE, seedling_color_confidence)
    
    # Update confidence
    # seedling_color_confidence = calculateBayes( seedling_color_confidence * (seedling_mask * PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT) * position_weight, (1 - seedling_color_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_mask * PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT) * (1 - position_weight) )
    seedling_color_confidence = calculateBayes( seedling_color_confidence * seedling_color_probability * position_weight, (1 - seedling_color_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_color_probability) * (1 - position_weight) )

    #  Calculate mean of seedling color confidence
    mean_seedling_color_confidence = np.mean(seedling_color_confidence)

    # Visualization
    # plt.subplot(100, 7, 3 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * seedling_mask[:,:,None]), plt.title('seedling')
    plt.subplot(100, 7, 3 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * seedling_mask[:,:,None]), plt.title('seedling')
    plt.subplot(100, 7, 4 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * (1 - seedling_mask[:,:,None])), plt.title('background')
    plt.subplot(100, 7, 5 + (7*(current_image_index-1))), plt.imshow(seedling_mask, cmap = 'jet', vmin=0, vmax=1), plt.title(f'Probability : mean = {seedling_mask.sum() / IMAGE_SHAPE.prod() :2f}'), plt.colorbar()
    plt.subplot(100, 7, 6 + (7*(current_image_index-1))), plt.imshow(seedling_color_confidence, cmap = 'jet', vmin=0, vmax=1), plt.title('Color Confidence'), plt.colorbar()
    plt.subplot(100, 7, 7 + (7*(current_image_index-1))), plt.hist(seedling_color_confidence.reshape(-1,1), bins=100, log=True, range=[0, 1]), plt.title('mean = ' + str(mean_seedling_color_confidence))
        
    # If the mean seedling color confidence is below the threshold, it indicates that there are not enough color values for initializing seedling color models.
    if mean_seedling_color_confidence < MEAN_SEEDLING_COLOR_CONFIDENCE_THRESHOLD:
        current_image_index += 1
    else:
        break
    
# Calculate initial seedling color model
init_mean_seedling_color, init_covariance_seedling_color = calculateMeanAndCovariance(current_image_color_vector, seedling_color_confidence)

# Fit a Gaussian Mixture Model to the background color values and initialize color model parameters
gmm = GaussianMixture(n_components = MAX_NUM_GAUSSIAN_MODELS - 1)
gmm.fit(ycbcr_vectors_list[0])
init_mean_background_color_models, init_covariance_background_color_models = gmm.means_, gmm.covariances_
init_color_model_means = np.vstack([init_mean_seedling_color, init_mean_background_color_models])
init_color_model_covariances = np.vstack([np.expand_dims(init_covariance_seedling_color, axis=0), init_covariance_background_color_models])
# Visualization
plot_ellipsoid([[init_mean_seedling_color, init_mean_background_color]], [[init_covariance_seedling_color, init_covariance_background_color]], ['Initial color model at '+ timestamp_list[current_image_index]])


## Update the seedling model from the day before the seedling color can be determined

Init single kernel

In [ ]:
# Create meshgrid of row and column indices based on tray dimensions
tray_row_indices, tray_col_indices = np.meshgrid(range(TRAY_ROW_COUNT), range(TRAY_COLUMN_COUNT))
# Stack row and column indices to form a 2D array representing cell indices in the tray
tray_cell_indices = np.stack([tray_row_indices.T, tray_col_indices.T], axis=-1)
# Calculate the centroid of each cell in the kernel
kernel_centroid = HALF_CELL_SHAPE + tray_cell_indices * CELL_SHAPE
# Calculate the top-left and bottom-right positions of the kernel for each cell
single_kernel_start = kernel_centroid - HALF_CELL_SHAPE
single_kernel_end = kernel_centroid + HALF_CELL_SHAPE + (CELL_SHAPE % 2)
# Adjust kernel positions to ensure they stay within image boundaries
single_kernel_start = np.where(single_kernel_start < 0, 0, single_kernel_start)
single_kernel_end = np.where(single_kernel_end < IMAGE_SHAPE, single_kernel_end, IMAGE_SHAPE)
# Create position vectors for each pixel in a cell (relative to the cell's center)
pixel_row_indices, pixel_col_indices = np.meshgrid(range(-HALF_CELL_HEIGHT, HALF_CELL_HEIGHT), range(-HALF_CELL_WIDTH, HALF_CELL_WIDTH))
pixel_positions_single_cell = np.stack([pixel_row_indices.T, pixel_col_indices.T], axis=-1)

Update

In [ ]:
# Create prior probability for target positions in the spatial domain to find seedling color vectors
prior_probability_position_all_cells = createPrior(INIT_COVARIANCE_PRIOR_SCALING, IMAGE_SHAPE, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
# Initialize prior probability for target positions in the spatial domain
# prior_probability_position_all_cells = np.zeros(IMAGE_SHAPE)
    
fig = plt.figure(figsize=(30,300))

for init_image_index in range(1, current_image_index + 1):
    
    current_image = ycbcr_images_list[init_image_index]
    current_image_color_vector = ycbcr_vectors_list[init_image_index]
    
    # Calculate the likelihood of each pixel belonging to the foreground (seedling) color model
    foreground_likelihood_all_cells = calculateMultiGaussianDist(current_image, init_color_model_means[0], init_color_model_covariances[0])

    # Initialize background likelihood as zero and accumulate likelihoods from background color models
    background_likelihood_all_cells = np.zeros(IMAGE_SHAPE)
    for index_background_clusters in range (1, MAX_NUM_GAUSSIAN_MODELS):
        background_likelihood_all_cells += calculateMultiGaussianDist(current_image, init_color_model_means[index_background_clusters], init_color_model_covariances[index_background_clusters])

    # Calculate seedling probability for all cells using Bayesian segmentation
    seedling_probability_all_cells = calculateBayes(foreground_likelihood_all_cells * prior_probability_position_all_cells, background_likelihood_all_cells * (SPATIAL_BACKGROUND_PRIOR_SCALING * prior_probability_position_all_cells.max() - prior_probability_position_all_cells) / (MAX_NUM_GAUSSIAN_MODELS - 1))
    
    # If there are enough seedling pixels to calculate mean and covariance matrix of seedling color
    if seedling_probability_all_cells.sum() > MIN_SEEDLING_PIXEL_IN_IMAGE_FOR_CALCULATION:
                
        # Calculate the mean and covariance for the seedling color model
        # mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image, seedling_probability_all_cells)    
        mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image[np.where(seedling_probability_all_cells > PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])    
        
        # Fit a Gaussian mixture model to the background pixels
        number_of_color_models_buffer[init_image_index] = MAX_NUM_GAUSSIAN_MODELS if init_image_index < current_image_index else MAX_NUM_GAUSSIAN_MODELS - 1
        gmm = GaussianMixture(n_components = number_of_color_models_buffer[init_image_index] - 1)
        gmm.fit(current_image[np.where(seedling_probability_all_cells < PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])
        mean_background_color, cov_background_color = gmm.means_, gmm.covariances_

        # Store the updated color model means and covariances
        color_model_means_buffer[init_image_index, :number_of_color_models_buffer[init_image_index]] = np.vstack([mean_seedling_color, mean_background_color])
        color_model_covariances_buffer[init_image_index, :number_of_color_models_buffer[init_image_index]] = np.vstack([np.expand_dims(cov_seedling_color, axis=0), cov_background_color])
                 
        # Iterate over each cell in the tray
        for row_index in range(TRAY_ROW_COUNT):
            for col_index in range(TRAY_COLUMN_COUNT):

                # Extract the seedling probability and image data for the current cell's kernel region
                kernel_seedling_probability = seedling_probability_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                    single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]].copy()
                kernel_image = current_image[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                                   single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]].copy()
                
                # Calculate and store the seedling size (sum of probabilities) and size-to-cell ratio
                seedling_size_buffer[init_image_index, row_index, col_index] = kernel_seedling_probability.sum()
                seedling_size_to_cell_ratio_buffer[init_image_index, row_index, col_index] = np.mean(kernel_seedling_probability)
                
                # If the size-to-cell ratio exceeds the threshold, the cell is considered to contain a seedling
                if seedling_size_to_cell_ratio_buffer[init_image_index, row_index, col_index] > SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD:
                    
                    seedling_status_buffer[init_image_index, row_index, col_index] = 1
                    
                    # Calculate the mean and covariance in the spatial domain for the seedling's position
                    seedling_position_model_means_buffer[init_image_index, row_index, col_index], seedling_position_model_covariances_buffer[init_image_index, row_index, col_index] = calculateMeanAndCovariance(pixel_positions_single_cell, kernel_seedling_probability)
                    
                    # Calculate and store the mean and covariance for the seedling's color
                    seedling_color_model_each_cell_means_buffer[init_image_index, row_index, col_index], seedling_color_model_each_cell_covariances_buffer[init_image_index, row_index, col_index] = calculateMeanAndCovariance(kernel_image, kernel_seedling_probability)
                    seedling_color_model_each_cell_means_buffer[init_image_index, row_index, col_index], seedling_color_model_each_cell_covariances_buffer[init_image_index, row_index, col_index] = calculateMeanAndCovariance(kernel_image, kernel_seedling_probability)

                    # Calculate the ellipse area based on the seedling's position covariance
                    ellipse_size_buffer[init_image_index, row_index, col_index] = calculateEllipseArea(seedling_position_model_covariances_buffer[init_image_index, row_index, col_index])
                
                # Update the prior probability when image index is current image index based on the current cell's seedling status
                if init_image_index == current_image_index:
                    if seedling_status_buffer[init_image_index, row_index, col_index] == 1:
                        prior_probability_position_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                            single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]] = calculateMultiGaussianDist(pixel_positions_single_cell, seedling_position_model_means_buffer[init_image_index, row_index, col_index], seedling_position_model_covariances_buffer[init_image_index, row_index, col_index] * COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING)
                    else:
                        prior_probability_position_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                            single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]] = calculateMultiGaussianDist(pixel_positions_single_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)
        
        if np.any(seedling_status_buffer[init_image_index]):
            
            # Calculate Euclidean distance between the seedling's center and the planting cell's center for each cell in each image
            # distance_position = np.sqrt(np.square(seedling_position_model_means_buffer[init_image_index]).sum(axis = -1))
            distance_position = calculateEuclidean(seedling_position_model_means_buffer[init_image_index], np.zeros_like(seedling_position_model_means_buffer[init_image_index]))
            distance_position_buffer[init_image_index] = np.where(seedling_status_buffer[init_image_index] == 0, 0, distance_position)
            # Calculate Euclidean distance between the mean Cb-Cr values of each seedling and the mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
            # distance_color= np.sqrt(np.square(seedling_color_model_each_cell_means_buffer[init_image_index, :, :, 1:] - color_model_means_buffer[init_image_index, 0, 1:]).sum(axis = -1))
            distance_color = calculateEuclidean(seedling_color_model_each_cell_means_buffer[init_image_index, :, :, 1:], color_model_means_buffer[init_image_index, 0, 1:])
            distance_color_buffer[init_image_index] = np.where(seedling_status_buffer[init_image_index] == 0, 0, distance_color)
            
            # Update confidence for seedling status based on previous confidence, as well as color and position distances
            seedling_status_confidence = np.where(((seedling_status_confidence_buffer[init_image_index - 1] < INIT_CONFIDENCE).astype(int) * (seedling_status_buffer[init_image_index]).astype(int)), INIT_CONFIDENCE, seedling_status_confidence_buffer[init_image_index - 1])
            
            seedling_status = np.where(seedling_status_buffer[init_image_index] == 1, STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT, 1 - STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT)
            
            # max_seedling_color_distance is the minimum Euclidean distance between the mean color of the seedling and the nearest background color model in YCbCr space
            # max_seedling_color_distance = (np.sqrt(np.square(color_model_means_buffer[init_image_index, 1:number_of_color_models_buffer[init_image_index]] - color_model_means_buffer[init_image_index, 0]).sum(axis=-1))).min()
            max_seedling_color_distance = calculateEuclidean(color_model_means_buffer[init_image_index, 1:number_of_color_models_buffer[init_image_index]], color_model_means_buffer[init_image_index, 0]).min()
            # Calculate color_distance_scaling using a linear equation: 
            # If the color distance is 0, scaling equals COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the color distance equals max_seedling_color_distance, scaling equals 1 - COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT.
            color_distance_scaling = COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_color_buffer[init_image_index] / max_seedling_color_distance )  
            color_distance_scaling = np.clip(np.where(seedling_status_buffer[init_image_index] == 0, 0.5, color_distance_scaling), a_max= COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT)

            # max_seedling_position_distance is the Euclidean distance from the center of the cell to the corner of the cell
            # max_seedling_position_distance = np.sqrt(np.square(HALF_CELL_SHAPE).sum())
            max_seedling_position_distance = calculateEuclidean(HALF_CELL_SHAPE, np.zeros_like(HALF_CELL_SHAPE))
            # Calculate position_distance_scaling using a linear equation:
            # If the position distance is 0, scaling equals POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the position distance equals max_seedling_position_distance, scaling equals 1 - POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT.
            position_distance_scaling = POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_position_buffer[init_image_index] / max_seedling_position_distance )
            position_distance_scaling = np.clip(np.where(seedling_status_buffer[init_image_index] == 0, 0.5, position_distance_scaling), a_max= POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT)
            
            # Update confidence
            foreground_probability = seedling_status_confidence * seedling_status * color_distance_scaling * position_distance_scaling
            background_probability = (1 - seedling_status_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_status) * (1 - color_distance_scaling) * (1 - position_distance_scaling)
            seedling_status_confidence_buffer[init_image_index] = calculateBayes( foreground_probability, background_probability )
            
            # Visualization
            ellipse_image = displayEllipseOnImage(resized_rgb_images_list[init_image_index], seedling_status_buffer[init_image_index], seedling_position_model_means_buffer[init_image_index], seedling_position_model_covariances_buffer[init_image_index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
            plt.subplot(100,7,((init_image_index-1)*7)+5), plt.imshow(ellipse_image), plt.title(f'num_gaussian={number_of_color_models_buffer[init_image_index]}')
            plt.subplot(100,7,((init_image_index-1)*7)+6), plt.imshow(seedling_status_confidence_buffer[init_image_index], vmax=1, vmin=0, cmap='hot'), plt.colorbar(), plt.title('seedling status confidence')
            plt.subplot(100,7,((init_image_index-1)*7)+7), plt.hist(seedling_status_confidence_buffer[init_image_index].reshape(-1,1), range=[0,1], log=True, bins=40), plt.title('seedling status confidence')
            
        # Visualization
        plt.subplot(100,7,((init_image_index-1)*7)+2), plt.imshow(seedling_probability_all_cells, cmap='hot'), plt.title(f'Probability Image (sum={seedling_probability_all_cells.mean():2f})'),
        plt.subplot(100,7,((init_image_index-1)*7)+1), plt.imshow(resized_rgb_images_list[init_image_index], cmap='hot'),plt.title(f'{init_image_index} : {timestamp_list[init_image_index]}')
        plt.subplot(100,7,((init_image_index-1)*7)+3), plt.imshow(resized_rgb_images_list[init_image_index] * seedling_probability_all_cells[:,:,None]), plt.title('Seedling')
        plt.subplot(100,7,((init_image_index-1)*7)+4), plt.imshow(resized_rgb_images_list[init_image_index] * (1-seedling_probability_all_cells[:,:,None])), plt.title('Background')

## Update the seedling model from the day after the seedling color can be determined

Init expanded kernel

In [ ]:
# init kernel for calculate seedling probability of each cell
# Create meshgrid of row and column indices based on tray dimensions
tray_row_indices, tray_col_indices = np.meshgrid(range(TRAY_ROW_COUNT), range(TRAY_COLUMN_COUNT))
# Stack row and column indices to form a 2D array representing cell indices in the tray
tray_cell_indices = np.stack([tray_row_indices.T, tray_col_indices.T], axis=-1)
# Calculate the centroid of each cell in the kernel
kernel_centroid = HALF_CELL_SHAPE + tray_cell_indices * CELL_SHAPE
# Calculate the top-left and bottom-right positions of the kernel for each cell
expanded_kernel_start = kernel_centroid - (HALF_CELL_SHAPE * 3)
expanded_kernel_end = kernel_centroid + (HALF_CELL_SHAPE * 3) + (CELL_SHAPE % 2)
# Adjust kernel positions to ensure they stay within image boundaries
expanded_kernel_start = np.where(expanded_kernel_start < 0, 0, expanded_kernel_start)
expanded_kernel_end = np.where(expanded_kernel_end < IMAGE_SHAPE, expanded_kernel_end, IMAGE_SHAPE)

Backup data

In [ ]:
# Backup data for rerunning the program
current_image_index_l = current_image_index
prior_probability_position_all_cells_l = prior_probability_position_all_cells.copy()

Update

In [ ]:
# Restore saved data for rerunning the process with previously backed-up state
current_image_index  = current_image_index_l
prior_probability_position_all_cells = prior_probability_position_all_cells_l.copy()

# Visualization
plt.figure(figsize=(35,600))

# Begin iterating through the remaining images
while(current_image_index < number_of_images - 1): 
# while(current_image_index < current_image_index_l+5): 
     
    current_image_index += 1
    
    # Retrieve current image and associated color data
    current_image = ycbcr_images_list[current_image_index]
    current_image_color_vector = ycbcr_vectors_list[current_image_index]
    # Retrieve Gaussian mixture model parameters from the buffer
    color_model_means_previous = color_model_means_buffer[current_image_index - 1,:number_of_color_models_buffer[current_image_index - 1]]
    color_model_covariances_previous = color_model_covariances_buffer[current_image_index - 1,:number_of_color_models_buffer[current_image_index - 1]]
    
    # Fit a Gaussian Mixture Model (GMM) to the background pixels of the current image
    gmm = GaussianMixture(n_components = number_of_color_models_buffer[current_image_index - 1])
    # gmm = GaussianMixture(n_components = number_of_color_models_buffer[current_image_index - 1], means_init = color_model_means_previous)
    gmm.fit(current_image_color_vector)
    gmm_means = gmm.means_
    gmm_covariances = gmm.covariances_
    
    # Align GMM models with existing color models, ensuring the seedling model is first.
    pair_gmm_indices = pairGaussianModels(color_model_means_previous, color_model_covariances_previous, gmm_means, gmm_covariances)
    gmm_means = gmm_means[pair_gmm_indices[1]]
    gmm_covariances = gmm_covariances[pair_gmm_indices[1]]

    # Compute likelihoods for the foreground (seedling) and background using color models and GMMs
    pdf_seedling_color_model = calculateMultiGaussianDist(current_image, color_model_means_previous[0], color_model_covariances_previous[0])
    pdf_seedling_gmm = calculateMultiGaussianDist(current_image, gmm_means[0], gmm_covariances[0])
    foreground_likelihood_all_cells = pdf_seedling_color_model * pdf_seedling_gmm
    
    background_likelihood_all_cells = np.zeros(IMAGE_SHAPE)
    for index_background_clusters in range (1, number_of_color_models_buffer[current_image_index - 1]):
        pdf_background_color_model = calculateMultiGaussianDist(current_image, color_model_means_previous[index_background_clusters], color_model_covariances_previous[index_background_clusters])
        pdf_background_gmm = calculateMultiGaussianDist(current_image, gmm_means[index_background_clusters], gmm_covariances[index_background_clusters])
        background_likelihood_all_cells += pdf_background_color_model * pdf_background_gmm 

    # Calculate the seedling probability for each cell using Bayes' theorem
    seedling_probability_all_cells = calculateBayes(foreground_likelihood_all_cells * prior_probability_position_all_cells, background_likelihood_all_cells * (SPATIAL_BACKGROUND_PRIOR_SCALING * prior_probability_position_all_cells.max() - prior_probability_position_all_cells) / (number_of_color_models_buffer[current_image_index - 1] - 1))
    
    
    # Compute the mean and covariance for seedling color pixels
    # mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image, seedling_probability_all_cells)    
    mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image[np.where(seedling_probability_all_cells > PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])    

    # Initialize an array to store the minimum Euclidean and Bhattacharyya distances for each number of Gaussian models
    seedling_color_gaussian_model_distance = np.zeros((2, number_of_color_models_buffer[current_image_index - 1] - MIN_NUM_GAUSSIAN_MODELS + 1))
    # seedling_color_gaussian_model_distance = np.zeros((3, number_of_color_models_buffer[current_image_index - 1] - MIN_NUM_GAUSSIAN_MODELS + 1))

    if number_of_color_models_buffer[current_image_index - 1] > MIN_NUM_GAUSSIAN_MODELS:
        # Iterate over the range of possible numbers of Gaussian models
        for n_gaussians in range(MIN_NUM_GAUSSIAN_MODELS, number_of_color_models_buffer[current_image_index - 1] + 1):
            
            # Initialize and fit the Gaussian Mixture Model with the current number of components (Gaussian models)
            gmm = GaussianMixture(n_components=n_gaussians)
            gmm.fit(current_image_color_vector)
            
            labeled_vector = gmm.predict(current_image_color_vector)
            
            # Initialize arrays to store both Euclidean and Bhattacharyya distances for each cluster (Gaussian component)
            euclidean_distance_buffer = np.zeros(n_gaussians)
            bhattacharyya_distance_buffer = np.zeros(n_gaussians)
            # mse_buffer = np.zeros(n_gaussians)
            
            # Calculate the Euclidean and Bhattacharyya distances between each cluster's color model and the seedling color model
            for cluster_index in range(n_gaussians):

                # cluster_mask = np.where(labeled_vector == cluster_index, 1, 0).reshape(IMAGE_SHAPE)
                # mse_buffer[cluster_index] = np.square(seedling_probability_all_cells - cluster_mask).mean()
                # mse_buffer[cluster_index] = np.where(seedling_probability_all_cells - cluster_mask > 0, 1, 0).sum()

                # Compute the Euclidean distance between the seedling and the current cluster
                # euclidean_distance_buffer[cluster_index] = np.sqrt(np.square(mean_seedling_color - gmm.means_[cluster_index]).sum())
                # euclidean_distance_buffer[cluster_index] = calculateEuclidean(color_model_means_buffer[current_image_index - 1, 0], gmm.means_[cluster_index]) 
                euclidean_distance_buffer[cluster_index] = calculateEuclidean(mean_seedling_color, gmm.means_[cluster_index]) 
                # Compute the Bhattacharyya distance between the seedling and the current cluster
                # bhattacharyya_distance_buffer[cluster_index] = calculateBhattacharyya(color_model_means_buffer[current_image_index - 1, 0], color_model_covariances_buffer[current_image_index - 2, 0], gmm.means_[cluster_index], gmm.covariances_[cluster_index])
                bhattacharyya_distance_buffer[cluster_index] = calculateBhattacharyya(mean_seedling_color, cov_seedling_color, gmm.means_[cluster_index], gmm.covariances_[cluster_index])

            # Store the minimum Euclidean and Bhattacharyya distances for the current number of Gaussian models
            seedling_color_gaussian_model_distance[0, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = euclidean_distance_buffer.min()
            seedling_color_gaussian_model_distance[1, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = bhattacharyya_distance_buffer.min()
            # seedling_color_gaussian_model_distance[2, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = mse_buffer.min()
            
        # Determine the optimal number of Gaussian models based on the minimum value between both Euclidean and Bhattacharyya distances
        number_of_color_models_buffer[current_image_index] = MIN_NUM_GAUSSIAN_MODELS + seedling_color_gaussian_model_distance.argmin(axis=1).min()
        # number_of_color_models_buffer[current_image_index] = MIN_NUM_GAUSSIAN_MODELS + seedling_color_gaussian_model_distance[:2].argmin(axis=1).min()

    else: 
        seedling_color_gaussian_model_distance += MIN_NUM_GAUSSIAN_MODELS
        number_of_color_models_buffer[current_image_index] = MIN_NUM_GAUSSIAN_MODELS


    # Fit a new GMM with the selected number of components (excluding the seedling component) to update the background color models
    gmm = GaussianMixture(n_components = number_of_color_models_buffer[current_image_index] - 1)
    gmm.fit(current_image[np.where(seedling_probability_all_cells < PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])
    mean_background_color, cov_background_color = gmm.means_, gmm.covariances_
    
    # Update the color model with the seedling and background GMM parameters for the current image
    color_model_means_buffer[current_image_index, :number_of_color_models_buffer[current_image_index]] = np.vstack([mean_seedling_color, mean_background_color])
    color_model_covariances_buffer[current_image_index, :number_of_color_models_buffer[current_image_index]] = np.vstack([np.expand_dims(cov_seedling_color, axis=0), cov_background_color])
    
    prior_probability_position_all_cells = np.zeros(IMAGE_SHAPE)
    
    # Adjusts seedling position model covariance matrices by limiting eigenvalue ratios
    seedling_position_model_eigenvalues, seedling_position_model_eigenvectors = np.linalg.eig(seedling_position_model_covariances_buffer[current_image_index - 1])    
    min_eigenvalues = np.min(seedling_position_model_eigenvalues, axis=-1)
    max_eigenvalues = np.max(seedling_position_model_eigenvalues, axis=-1)
    seedling_major_minor_axis_ratio = (2.25 - 0.0042 * ellipse_size_buffer[current_image_index - 1]).clip(1.2, 2)
    adjusted_max_eigenvalues = np.where(max_eigenvalues > seedling_major_minor_axis_ratio * min_eigenvalues, seedling_major_minor_axis_ratio * min_eigenvalues, max_eigenvalues)
    adjusted_seedling_position_model_eigenvalues = np.where(seedling_position_model_eigenvalues == max_eigenvalues[..., None], adjusted_max_eigenvalues[..., None], seedling_position_model_eigenvalues)
    diagonal_matrices = adjusted_seedling_position_model_eigenvalues[..., None] * np.eye(adjusted_seedling_position_model_eigenvalues.shape[-1])
    adjusted_seedling_position_model_covariance = np.matmul(seedling_position_model_eigenvectors, np.matmul(diagonal_matrices, seedling_position_model_eigenvectors.transpose(0, 1, 3, 2)))

    # Scale the covariance matrix based on the size of the seedling ellipse
    covariance_scaling = (MAX_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING - COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING_RATIO * 1e-08 * ellipse_size_buffer[current_image_index - 1] ** 2).clip(MIN_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING, MAX_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING)
    
    # # Iterate over each cell in the tray
    # for row_index in range(TRAY_ROW_COUNT):
    #     for col_index in range(TRAY_COLUMN_COUNT):
    # cell_coordinates = [np.stack(np.where(seedling_status_buffer[current_image_index - 1]), axis=-1)]
    cell_coordinates = [np.stack(np.where(seedling_status_buffer[current_image_index - 1]), axis=-1), np.stack(np.where(1 - seedling_status_buffer[current_image_index - 1]), axis=-1)]
    for status_index in range(len(cell_coordinates)):
        if (status_index == 0 and len(cell_coordinates[status_index]) > 0) or (status_index == 1 and seedling_probability_remaining.sum() > 0):
            for row_index, col_index in cell_coordinates[status_index]:
                
                # Calculate the boundaries of each pixel relative to the cell's center
                relative_top_left = expanded_kernel_start[row_index, col_index] - kernel_centroid[row_index, col_index]
                relative_bottom_right  = expanded_kernel_end[row_index, col_index] - kernel_centroid[row_index, col_index] 
                # Generate position vectors for each pixel within the cell
                pixel_row_indices, pixel_col_indices = np.meshgrid(range(relative_top_left[0], relative_bottom_right[0]), range(relative_top_left[1], relative_bottom_right[1]))
                pixel_positions_in_cell = np.stack([pixel_row_indices.T, pixel_col_indices.T], axis=-1)
                
                # Create prior probability in spatial domain based on the seedling status from the previous image
                if seedling_status_buffer[current_image_index - 1, row_index, col_index] == 1:

                    # Calculate the prior probability of pixel positions in the cell based on a Gaussian distribution
                    prior_probability_position_in_cell = calculateMultiGaussianDist(pixel_positions_in_cell, seedling_position_model_means_buffer[current_image_index - 1, row_index, col_index], adjusted_seedling_position_model_covariance[row_index, col_index] * covariance_scaling[row_index, col_index])
                    
                    # Get foreground and background likelihoods based on seedling probability in the current cell
                    foreground_likelihood_in_cell = seedling_probability_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0], expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]]
                    background_likelihood_in_cell = 1 - foreground_likelihood_in_cell
                                    
                    kernel_image = current_image[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0], expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]]
                
                elif seedling_status_buffer[current_image_index - 1, row_index, col_index] == 0:
                    
                    # Prior probability for an empty cell is based on a default Gaussian distribution
                    prior_probability_position_in_cell = calculateMultiGaussianDist(pixel_positions_single_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)
                
                    # Get foreground and background likelihoods for an empty cell
                    foreground_likelihood_in_cell = seedling_probability_remaining[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0], single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]]
                    background_likelihood_in_cell = 1 - foreground_likelihood_in_cell
                    

                    kernel_image = current_image[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0], single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]]
    
                # Calculate the seedling probability in the cell using Bayesian segmentation
                seedling_probability_each_cell = calculateBayes(foreground_likelihood_in_cell * prior_probability_position_in_cell , background_likelihood_in_cell * (SPATIAL_BACKGROUND_PRIOR_SCALING * prior_probability_position_in_cell.max() - prior_probability_position_in_cell) / (number_of_color_models_buffer[current_image_index] - 1))
                
                # Calculate seedling size as a ratio to the cell size and update the buffer
                seedling_size_to_cell_ratio_buffer[current_image_index, row_index, col_index] = seedling_probability_each_cell.sum() / CELL_SHAPE.prod()
                seedling_size_buffer[current_image_index, row_index, col_index] = seedling_probability_each_cell.sum()


                # Check if seedling size exceeds the threshold
                if seedling_size_to_cell_ratio_buffer[current_image_index, row_index, col_index] > SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD:

                    seedling_status_buffer[current_image_index, row_index, col_index] = 1

                    # Update position means and covariances based on seedling status
                    if seedling_status_buffer[current_image_index - 1, row_index, col_index] == 1: 
                        position_means, position_covariances = calculateMeanAndCovariance(pixel_positions_in_cell, seedling_probability_each_cell)

                    elif seedling_status_buffer[current_image_index - 1, row_index, col_index] == 0: 
                        position_means, position_covariances = calculateMeanAndCovariance(pixel_positions_single_cell, seedling_probability_each_cell)

                else: 
                    # If seedling size is below threshold, flag the cell as not having started germination
                    seedling_status_buffer[current_image_index, row_index, col_index] = 0

                # If seedling is present in the current image, update the position and covariance buffers
                if seedling_status_buffer[current_image_index, row_index, col_index] == 1:
                    
                    seedling_position_model_means_buffer[current_image_index, row_index, col_index], seedling_position_model_covariances_buffer[current_image_index, row_index, col_index] = position_means.copy(), position_covariances.copy()
                    # Update the prior probability in the spatial domain based on the current Gaussian distribution
                    prior_probability_position_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0],
                                                    expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]] += calculateMultiGaussianDist(pixel_positions_in_cell, seedling_position_model_means_buffer[current_image_index, row_index, col_index], seedling_position_model_covariances_buffer[current_image_index, row_index, col_index] * COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING)
                    # Update the ellipse size buffer and color model for the cell
                    ellipse_size_buffer[current_image_index, row_index, col_index] = calculateEllipseArea(seedling_position_model_covariances_buffer[current_image_index, row_index, col_index])
                    seedling_color_model_each_cell_means_buffer[current_image_index, row_index, col_index], seedling_color_model_each_cell_covariances_buffer[current_image_index, row_index, col_index] = calculateMeanAndCovariance(kernel_image, seedling_probability_each_cell)
                    
                else:
                    # If no seedling is present, update the prior probability using an empty cell model
                    prior_probability_position_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0],
                                    expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]] += calculateMultiGaussianDist(pixel_positions_in_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)
            if status_index == 0:
                assigned_seedling_masks = createSeedlingEllipseMask(IMAGE_SHAPE, seedling_status_buffer[current_image_index], seedling_position_model_means_buffer[current_image_index], seedling_position_model_covariances_buffer[current_image_index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
                seedling_probability_remaining = ( 1 - assigned_seedling_masks) * seedling_probability_all_cells
        elif status_index == 0 and len(cell_coordinates[status_index]) > 0 : 
            seedling_probability_remaining = seedling_probability_all_cells.copy()

    # Calculate Euclidean distance between the seedling's center and the planting cell's center for each cell in each image
    # distance_position = np.sqrt(np.square(seedling_position_model_means_buffer[current_image_index]).sum(axis = -1))
    distance_position = calculateEuclidean(seedling_position_model_means_buffer[current_image_index], np.zeros_like(seedling_position_model_means_buffer[current_image_index]))
    distance_position_buffer[current_image_index] = np.where(seedling_status_buffer[current_image_index] == 0, 0, distance_position)
    # Calculate Euclidean distance between the mean Cb-Cr values of each seedling and the mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
    # distance_color= np.sqrt(np.square(seedling_color_model_each_cell_means_buffer[current_image_index, :, :, 1:] - color_model_means_buffer[current_image_index, 0, 1:]).sum(axis = -1))
    distance_color = calculateEuclidean(seedling_color_model_each_cell_means_buffer[current_image_index, :, :, 1:], color_model_means_buffer[current_image_index, 0, 1:])
    distance_color_buffer[current_image_index] = np.where(seedling_status_buffer[current_image_index] == 0, 0, distance_color)
    
    # Update confidence for seedling status based on previous confidence, as well as color and position distances
    seedling_status_confidence = np.where(((seedling_status_confidence_buffer[current_image_index - 1] < INIT_CONFIDENCE).astype(int) * (seedling_status_buffer[current_image_index]).astype(int)), INIT_CONFIDENCE, seedling_status_confidence_buffer[current_image_index - 1])
    seedling_status = np.where(seedling_status_buffer[current_image_index] == 1, STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT, 1 - STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # max_seedling_color_distance is the minimum Euclidean distance between the mean color of the seedling and the nearest background color model in YCbCr space
    max_seedling_color_distance = calculateEuclidean(color_model_means_buffer[current_image_index, 1:number_of_color_models_buffer[current_image_index]], color_model_means_buffer[current_image_index, 0]).min()
    # Calculate color_distance_scaling using a linear equation: 
    # If the color distance is 0, scaling equals COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the color distance equals max_seedling_color_distance, scaling equals 1 - COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT.
    color_distance_scaling = COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_color_buffer[current_image_index] / max_seedling_color_distance )  
    color_distance_scaling = np.clip(np.where(seedling_status_buffer[current_image_index] == 0, 0.5, color_distance_scaling), a_max= COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # max_seedling_position_distance is the Euclidean distance from the center of the cell to the corner of the cell
    # max_seedling_position_distance = np.sqrt(np.square(HALF_CELL_SHAPE).sum())
    max_seedling_position_distance = calculateEuclidean(HALF_CELL_SHAPE, np.zeros_like(HALF_CELL_SHAPE))
    # Calculate position_distance_scaling using a linear equation:
    # If the position distance is 0, scaling equals POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the position distance equals max_seedling_position_distance, scaling equals 1 - POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT.
    position_distance_scaling = POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_position_buffer[current_image_index] / max_seedling_position_distance )
    position_distance_scaling = np.clip(np.where(seedling_status_buffer[current_image_index] == 0, 0.5, position_distance_scaling), a_max= POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # Update confidence
    foreground_probability = seedling_status_confidence * seedling_status * color_distance_scaling * position_distance_scaling
    background_probability = (1 - seedling_status_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_status) * (1 - color_distance_scaling) * (1 - position_distance_scaling)
    seedling_status_confidence_buffer[current_image_index] = calculateBayes( foreground_probability, background_probability )

    # Visualization
    ellipse_image = displayEllipseOnImage(resized_rgb_images_list[current_image_index].copy(), seedling_status_buffer[current_image_index], seedling_position_model_means_buffer[current_image_index], seedling_position_model_covariances_buffer[current_image_index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
    # title_num_gaussian = f'num_gaussian={number_of_color_models_buffer[current_image_index]}' if number_of_color_models_buffer[current_image_index] == number_of_color_models_buffer[current_image_index-1] else f'num_gaussian={number_of_color_models_buffer[current_image_index]} (e,b = {MIN_NUM_GAUSSIAN_MODELS+seedling_color_gaussian_model_distance.argmin(axis=1)})'
    title_num_gaussian = f'num_gaussian={number_of_color_models_buffer[current_image_index]} (n = {MIN_NUM_GAUSSIAN_MODELS+seedling_color_gaussian_model_distance.argmin(axis=1)})'
    # plt.subplot(150,5,5*(current_image_index - current_image_index_l)-2), plt.imshow(ellipse_image), plt.title(title_num_gaussian)
    plt.subplot(150,5,5*(current_image_index - current_image_index_l)-2), plt.imshow(ellipse_image), plt.title(title_num_gaussian)
    plt.subplot(150,5,5*(current_image_index - current_image_index_l)-1), plt.imshow(seedling_status_confidence_buffer[current_image_index], vmax=1, vmin=0, cmap='hot'), plt.colorbar(), plt.title('seedling status confidence')
    plt.subplot(150,5,5*(current_image_index - current_image_index_l)),   plt.hist(seedling_status_confidence_buffer[current_image_index].reshape(-1,1), range=[0,1], log=True, bins=40), plt.title('seedling status confidence')
    # plt.subplot(150,5,5*(current_image_index - current_image_index_l)),   plt.imshow(seedling_probability_remaining, 'hot'), plt.title(f'seedling_probability_remaining')
    plt.subplot(150,5,5*(current_image_index - current_image_index_l)-3), plt.imshow(seedling_probability_all_cells, 'hot'), plt.title(f'Prob : mean={seedling_probability_all_cells.mean()}')
    plt.subplot(150,5,5*(current_image_index - current_image_index_l)-4), plt.imshow(resized_rgb_images_list[current_image_index]), plt.title(f'{current_image_index} : {timestamp_list[current_image_index]}')

color model

In [ ]:
plot_ellipsoid_color_model(color_model_means_buffer,  color_model_covariances_buffer, timestamp_list)


## Report

#####  ellipse overlay

In [ ]:
fig = plt.figure(figsize=(25,200))
for index in range(number_of_images):
    ellipse_image = displayEllipseOnImage(resized_rgb_images_list[index].copy(), seedling_status_buffer[index], seedling_position_model_means_buffer[index], seedling_position_model_covariances_buffer[index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
    plt.subplot(50,3,1+index)
    plt.imshow(ellipse_image), plt.title(timestamp_list[index].split(".")[0])


In [ ]:
n = 3
plt.figure(figsize=(30,10*n))
plt.subplot(n,1,1), plt.plot(timestamp_list, np.where(seedling_status_confidence_buffer > SEEDLING_STATUS_CONFIDENCE_THRESHOLD, 1, 0).sum(-1).sum(-1)), plt.title('mean seedling_status_confidence'), plt.ylim(0, TRAY_ROW_COUNT * TRAY_COLUMN_COUNT), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
# plt.subplot(n,1,2), plt.plot(timestamp_list, seedling_status_buffer.mean(axis=-1).mean(axis=-1)), plt.title('mean seedling_status'), plt.ylim(0, 1), plt.grid(linewidth = "0.1")
plt.subplot(n,1,2)
plt.plot(timestamp_list, ellipse_size_buffer.reshape(number_of_images, -1).mean(axis = -1))
# plt.plot(timestamp_list, seedling_size_buffer.reshape(number_of_images, -1).mean(axis = -1), label = 'mean seedling_size')
plt.title('mean size'), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
plt.subplot(n,3,3*3-2), plt.plot(timestamp_list, color_model_means_buffer[:,0,0]), plt.title('Y'), plt.ylim(0, 1), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
plt.subplot(n,3,3*3-1), plt.plot(timestamp_list, color_model_means_buffer[:,0,1]), plt.title('Cb'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
plt.subplot(n,3,3*3), plt.plot(timestamp_list, color_model_means_buffer[:,0,2]), plt.title('Cr'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)


In [ ]:
def calculateMeanByDate(timestampList, variableList):
    # Extract unique dates
    uniqueDates = sorted(set(timestamp.split('_')[0] for timestamp in timestampList))
    
    # Initialize a dictionary to store results
    dateMeans = {}
    
    # Loop through each unique date
    for date in uniqueDates:
        # Find indices of timestamps that match the current date
        indices = [i for i, timestamp in enumerate(timestampList) if timestamp.startswith(date)]
        
        # Get the corresponding values from the variable list
        values = [variableList[i] for i in indices]
        
        # Calculate the mean
        dateMeans[date] = np.mean(values)
    
    return dateMeans

In [ ]:
n = 3
plt.figure(figsize=(30,10*n))

seedling_status_date = calculateMeanByDate(timestamp_list, np.where(seedling_status_confidence_buffer > SEEDLING_STATUS_CONFIDENCE_THRESHOLD, 1, 0).sum(-1).sum(-1))
plt.subplot(n,1,1), plt.plot(list(seedling_status_date.keys()), list(seedling_status_date.values())), plt.title('mean number of seedling'), plt.ylim(0, TRAY_ROW_COUNT * TRAY_COLUMN_COUNT), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
# plt.subplot(n,1,2), plt.plot(timestamp_list, seedling_status_buffer.mean(axis=-1).mean(axis=-1)), plt.title('mean seedling_status'), plt.ylim(0, 1), plt.grid(linewidth = "0.1")
plt.subplot(n,1,2)
ellipse_size_date = calculateMeanByDate(timestamp_list, ellipse_size_buffer.reshape(number_of_images, -1).mean(axis = -1))
plt.plot(ellipse_size_date.keys(), ellipse_size_date.values())
# plt.plot(timestamp_list, seedling_size_buffer.reshape(number_of_images, -1).mean(axis = -1), label = 'mean seedling_size')
plt.title('mean size'), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
y_color_model_means_date = calculateMeanByDate(timestamp_list, color_model_means_buffer[:,0,0])
plt.subplot(n,3,3*3-2), plt.plot(y_color_model_means_date.keys(), y_color_model_means_date.values()), plt.title('Y'), plt.ylim(0, 1), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
cb_color_model_means_date = calculateMeanByDate(timestamp_list, color_model_means_buffer[:,0,1])
plt.subplot(n,3,3*3-1), plt.plot(cb_color_model_means_date.keys(), cb_color_model_means_date.values()), plt.title('Cb'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)
cr_color_model_means_date = calculateMeanByDate(timestamp_list, color_model_means_buffer[:,0,2])
plt.subplot(n,3,3*3), plt.plot(cr_color_model_means_date.keys(), cr_color_model_means_date.values()), plt.title('Cr'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1"), plt.xticks(rotation=45)


In [ ]:
plt.figure(figsize=(30,8))
plt.subplot(2,5,1), plt.imshow(resized_rgb_images_list[current_image_index]),  plt.title(f'index = {current_image_index}, {timestamp_list[current_image_index]}')
plt.subplot(2,5,6), plt.imshow(ellipse_image),  plt.title(f'ellipse')
plt.subplot(2,5,2), plt.imshow(seedling_status_confidence_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'seedling status confidence')
plt.subplot(2,5,7), plt.hist(seedling_status_confidence_buffer[current_image_index].reshape(-1,1), range=[0,1], log=True, bins=40)
plt.subplot(2,5,3), plt.imshow(seedling_size_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'seedling size')
plt.subplot(2,5,8), plt.hist(seedling_size_buffer[current_image_index].reshape(-1,1), log=True, bins=40)
plt.subplot(2,5,4), plt.imshow(distance_color_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'euclidean distance between mean cb-cr of each seedling \nand mean cb-cr of entire tray in ycbcr color space')
plt.subplot(2,5,9), plt.hist(distance_color_buffer[current_image_index].reshape(-1,1), log=True, bins=40)
plt.subplot(2,5,5), plt.imshow(distance_position_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'euclidean distance between seedling center and planting cell center')
plt.subplot(2,5,10), plt.hist(distance_position_buffer[current_image_index].reshape(-1,1), log=True, bins=40)


In [ ]:
status_confidence_over_threshold = seedling_status_confidence_buffer[current_image_index] > SEEDLING_STATUS_CONFIDENCE_THRESHOLD

# Cells with seedling probability over the threshold
over_threshold_count = np.count_nonzero(status_confidence_over_threshold)
over_threshold_percentage = over_threshold_count / (TRAY_COLUMN_COUNT * TRAY_ROW_COUNT)
over_threshold_cells = np.stack(np.where(status_confidence_over_threshold), axis=-1)

# Cells with seedling probability under the threshold
under_threshold_count = np.count_nonzero(~status_confidence_over_threshold)
under_threshold_percentage = under_threshold_count / (TRAY_COLUMN_COUNT * TRAY_ROW_COUNT)
under_threshold_cells = np.stack(np.where(~status_confidence_over_threshold), axis=-1)

# Seedling size analysis (Outliers)
size_outliers = checkOutlier(status_confidence_over_threshold, seedling_size_buffer[current_image_index])
mean_size = seedling_size_buffer[current_image_index][status_confidence_over_threshold].mean() * PIXELS_TO_REAL_SIZE_RATIO

# Seedling color analysis (Outliers)
color_outliers = checkOutlier(status_confidence_over_threshold, distance_color_buffer[current_image_index])
mean_color_rgb = convertYCbCrtoRGB(color_model_means_buffer[current_image_index][0])

# Generate report-like output
print(f"Seedling Probability")
print(f"Cells with Seedling Probability > {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}:")
print(f"  Total: {over_threshold_count} cells ({over_threshold_percentage:.2%})")
print(f"  Locations: {', '.join(str(k) for k in over_threshold_cells)}")

print(f"\nCells with Seedling Probability ≤ {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}:")
print(f"  Total: {under_threshold_count} cells ({under_threshold_percentage:.2%})")
print(f"  Locations: {', '.join(str(k) for k in under_threshold_cells)}")

print("\nSeedling Size ")
print(f"  Mean Seedling Size: {mean_size:.2f} mm²")
print(f"  Size outliers detected at cells: {', '.join(str(k) for k in size_outliers)}")

print("\nSeedling Color ")
print(f"  Mean Seedling Color (RGB): {mean_color_rgb}")
print(f"  Color outliers detected at cells: {', '.join(str(k) for k in color_outliers)}")

plt.figure(figsize=(22,10))
plt.subplot(231), plt.imshow(resized_rgb_images_list[current_image_index]),  plt.title(f'{timestamp_list[current_image_index]}')
plt.subplot(232), plt.imshow(ellipse_image),  plt.title(f'Ellipse overlay')
plt.subplot(233), plt.imshow(status_confidence_over_threshold, 'gray'),  plt.title(f'Cells with Seedling Probability > {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}: {over_threshold_count} cells ({over_threshold_percentage:.2%})\nCells with Seedling Probability ≤ {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}: {under_threshold_count} cells ({under_threshold_percentage:.2%})', loc='left')
plt.subplot(234), plt.imshow(seedling_size_buffer[current_image_index], cmap='gray'), plt.colorbar(), plt.title(f'seedling size')
error_image = np.zeros((TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
error_image[size_outliers[:, 0], size_outliers[:, 1]] = 1
plt.subplot(235), plt.imshow(error_image, 'gray'),  plt.title(f'Mean Seedling Size: {mean_size:.2f} mm²\nSize outliers')
error_image = np.zeros((TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
error_image[color_outliers[:, 0], color_outliers[:, 1]] = 1
plt.subplot(236), plt.imshow(error_image, 'gray'),  plt.title(f'Mean Seedling Color (RGB): [{mean_color_rgb[0]:.2f}, {mean_color_rgb[1]:.2f}, {mean_color_rgb[2]:.2f}]\nColor outliers')


In [ ]:

## seedling_status_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : สถานะต้นกล้าในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, สถานะ) 0 ไม่มี 1 มี

## seedling_position_model_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2)) : mean ตำแหน่งต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, vector(2))
## seedling_position_model_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2, 2)) : covariance matrix ตำแหน่งต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, matrix(2,2))

# number_of_color_models_buffer = np.zeros(number_of_images, dtype=int) : จำนวน cluster ของ color model ในแต่ละภาพ (ภาพที่, จำนวน)

# color_model_means_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3)) : mean ของ color model ของต้นกล้าและพื้นหลังในแต่ละภาพ (ภาพที่, โมเดลสีที่, vector(3)) โมเดลสีแรกคือโมเดลสีผัก โมเดลสีที่เกินจากจำนวนโมเดลสีในแต่ละภาพมีค่าเป็น 0
# color_model_covariances_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3, 3)) : covariance matrix ของ color model ของต้นกล้าและพื้นหลังในแต่ละภาพ (ภาพที่, โมเดลสีที่, matrix(3,3)) โมเดลสีแรกคือโมเดลสีผัก โมเดลสีที่เกินจากจำนวนโมเดลสีในแต่ละภาพมีค่าเป็น 0

# seedling_color_model_each_cell_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3)) : mean ของ color model ของต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, vector(3)) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0
# seedling_color_model_each_cell_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3, 3)) : covariance matrix ของ color model ของต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, matrix(3,3)) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0

# ellipse_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ขนาดต้นกล้าแต่ละหลุมในแต่ละภาพจากการหาพื้นที่วงรี (ภาพที่, แถวที่, หลักที่, ขนาด) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0
# seedling_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ขนาดต้นกล้าแต่ละหลุมในแต่ละภาพจาก sum probability (ภาพที่, แถวที่, หลักที่, ขนาด)
# seedling_size_to_cell_ratio_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : อัตราส่วนขนาดต้นกล้าแต่ละหลุมแต่ขนาดหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, อัตราส่วน)

# distance_position_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ระยะห่างระหว่างตำแหน่งต้นกล้ากับตำแหน่งจุดศูนย์กลางหลุมปลูก (ภาพที่, แถวที่, หลักที่, ระยะห่าง)
# distance_color_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ระยะห่างระหว่างค่าสีต้นกล้าแต่ละหลุมกับค่าเฉลี่ยค่าสีต้นกล้าของทั้งถาด (ภาพที่, แถวที่, หลักที่, ระยะห่าง)

# seedling_status_confidence_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ความเชื่อมั่นที่แต่ละหลุมจะมีต้นกล้า (ภาพที่, แถวที่, หลักที่, ค่าความเชื่อมั่น)

In [ ]:
# status_confidence_over_threshold : สถานะต้นกล้าที่ผ่านเกณฑ์ความเชื่อมั่นที่ (ภาพที่, แถวที่, หลักที่, สถานะ)
# over_threshold_cells : หลุมที่มีต้นกล้า (n, 2)
# under_threshold_cells : หลุมที่ไม่มีต้นกล้า (n, 2)

# size_outliers : หลุมที่มีขนาดผิดปกติ (n, 2)
# mean_size : ค่าเฉลี่ยขนาดต้นกล้าในหน่วยตารางมิลลิเมตร

# color_outliers : หลุมที่มีสีผิดปกติ (n, 2)
# mean_color_rgb = ค่าเฉลี่ยสีต้นกล้า RGB